In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
df_countylist_input = pd.read_csv('County_List.csv')

### 1. ASC Base Year Population

In [2]:
df_pop_2019_county_input = pd.read_csv('Input_2019pop_allcounty_ASC.csv', encoding='Latin-1')

In [3]:
df_pop_2019_county = pd.merge(df_pop_2019_county_input, df_countylist_input, left_on = ['STNAME','CTYNAME'], right_on = ['StateCompleteName', 'NAMELSAD'])
df_pop_2019_county = df_pop_2019_county[['ZoneID','STNAME','CTYNAME','POPESTIMATE2019']]

In [4]:
df_pop_2019_county.to_csv('Output_2019pop_WisDOT_ASC.csv')

### 2. BEA Base Year Population/Employment/Income

In [5]:
df_emp_2019_county_input = pd.read_csv('Input_CAINC4__ALL_AREAS_1969_2020.csv', encoding='Latin-1', low_memory=False)

In [6]:
df_emp_2019_county_filter = df_emp_2019_county_input.loc[df_emp_2019_county_input['LineCode'].isin([20, 30, 7010])]
df_emp_2019_county_filter = df_emp_2019_county_filter[['GeoName', 'LineCode', 'Description', 'Unit', '2019']]
df_emp_2019_county_filter.GeoName = df_emp_2019_county_filter.GeoName.str.replace('*', '', regex=True) # CAUTION!

In [7]:
df_emp_2019_county = pd.merge(df_emp_2019_county_filter, df_countylist_input, left_on = 'GeoName', right_on = 'BEAName')
df_emp_2019_county = df_emp_2019_county[['ZoneID','GeoName', 'LineCode', 'Description', 'Unit', '2019']]

In [8]:
df_emp_2019_county_popout = df_emp_2019_county.loc[df_emp_2019_county.LineCode==20]
df_emp_2019_county_incout = df_emp_2019_county.loc[df_emp_2019_county.LineCode==30]
df_emp_2019_county_empout = df_emp_2019_county.loc[df_emp_2019_county.LineCode==7010]

In [9]:
df_emp_2019_county_empout.to_csv('Output_2019emp_WisDOT_BEA.csv')
df_emp_2019_county_popout.to_csv('Output_2019pop_WisDOT_BEA.csv')
df_emp_2019_county_incout.to_csv('Output_2019inc_WisDOT_BEA.csv')

### 3. Future Year (2025 Population) - Should be edited based on different sources

In [ ]:
df_pop_2035_Chicago_county_input = pd.read_csv('Input_2035pop_Chicago_County.csv')
df_pop_2035_Indiana_county_input = pd.read_csv('Input_2035pop_Indiana_County.csv')
df_pop_2035_Minnesota_county_input = pd.read_csv('Input_2035pop_Minnesota_County.csv')
df_pop_2035_Michigan_county_input = pd.read_csv('Input_2035pop_Michigan_County.csv') # Delete the first row to avoid parse error 
df_pop_2035_Wisconsin_county_input = pd.read_csv('Input_2035pop_Wisconsin_County.csv')

In [ ]:
df_pop_2035_Chicago_county = df_pop_2035_Chicago_county_input[['Head', '2035']]
df_pop_2035_Chicago_county = df_pop_2035_Chicago_county.assign(Fullname = df_pop_2035_Chicago_county.Head.astype(str) + ', IL')
df_pop_2035_Chicago_county = df_pop_2035_Chicago_county.drop([0, 8, 9, 10, 11, 12]).rename(columns={'Head': 'County', '2035' : '2035pop'}).reset_index(drop=True)

In [ ]:
df_pop_2035_Indiana_county = df_pop_2035_Indiana_county_input.iloc[:, 0:2]
df_pop_2035_Indiana_county = df_pop_2035_Indiana_county.drop(0).rename(columns={'Unnamed: 0': 'County', 'Unnamed: 1': '2035pop'})
df_pop_2035_Indiana_county = df_pop_2035_Indiana_county.assign(Fullname = df_pop_2035_Indiana_county.County.astype(str) + ', IN').reset_index(drop=True)

In [ ]:
df_pop_2035_Minnesota_county = df_pop_2035_Minnesota_county_input.loc[(df_pop_2035_Minnesota_county_input.year == 2035) & (df_pop_2035_Minnesota_county_input.geo_type > 2)]
df_pop_2035_Minnesota_county = df_pop_2035_Minnesota_county[['geo_id', '999']].rename(columns={'geo_id': 'County', '999': '2035pop'})
df_pop_2035_Minnesota_county = df_pop_2035_Minnesota_county.assign(Fullname = df_pop_2035_Minnesota_county.County.astype(str) + ' County, MN').reset_index(drop=True)

In [ ]:
df_pop_2035_Michigan_county = df_pop_2035_Michigan_county_input[['Area', '2035']]
df_pop_2035_Michigan_county = df_pop_2035_Michigan_county.rename(columns={'Area': 'Fullname', '2035': '2035pop'}).reset_index(drop=True)
df_pop_2035_Michigan_county = df_pop_2035_Michigan_county.assign(County = df_pop_2035_Michigan_county.Fullname.str[:-4])

In [ ]:
df_pop_2035_Wisconsin_county_input.columns = df_pop_2035_Wisconsin_county_input.iloc[1]
df_pop_2035_Wisconsin_county = df_pop_2035_Wisconsin_county_input[['County Name', 'PopAll 2035']].drop([0, 1])
df_pop_2035_Wisconsin_county = df_pop_2035_Wisconsin_county.assign(Fullname = df_pop_2035_Wisconsin_county[['County Name']].astype(str) + 'County, WI')
df_pop_2035_Wisconsin_county = df_pop_2035_Wisconsin_county.assign(Fullname = df_pop_2035_Wisconsin_county.Fullname.str[1:])
df_pop_2035_Wisconsin_county = df_pop_2035_Wisconsin_county.rename(columns={'County Name': 'County', 'PopAll 2035': '2035pop'}).reset_index(drop=True)

In [ ]:
df_pop_2035_county_input = pd.concat([df_pop_2035_Chicago_county, df_pop_2035_Indiana_county, df_pop_2035_Minnesota_county, df_pop_2035_Michigan_county, df_pop_2035_Wisconsin_county], ignore_index=True)
df_pop_2035_county_input.Fullname = df_pop_2035_county_input.Fullname.str.replace('Saint', 'St.', regex=True) # CAUTION!
df_pop_2035_county = pd.merge(df_pop_2035_county_input, df_countylist_input, on = 'Fullname')
df_pop_2035_county = df_pop_2035_county[['ZoneID','Fullname','2035pop']]
df_pop_2035_county.to_csv('Output_2035pop_WisDOT.csv')

### 4. Future Year (2035 Population) - Should be edited based on different sources

In [10]:
df_pop_2035_Chicago_county_input = pd.read_csv('Input_2035pop_Chicago_County.csv')
df_pop_2035_Indiana_county_input = pd.read_csv('Input_2035pop_Indiana_County.csv')
df_pop_2035_Minnesota_county_input = pd.read_csv('Input_2035pop_Minnesota_County.csv')
df_pop_2035_Michigan_county_input = pd.read_csv('Input_2035pop_Michigan_County.csv') # Delete the first row to avoid parse error 
df_pop_2035_Wisconsin_county_input = pd.read_csv('Input_2035pop_Wisconsin_County.csv')

In [11]:
df_pop_2035_Chicago_county = df_pop_2035_Chicago_county_input[['Head', '2035']]
df_pop_2035_Chicago_county = df_pop_2035_Chicago_county.assign(Fullname = df_pop_2035_Chicago_county.Head.astype(str) + ', IL')
df_pop_2035_Chicago_county = df_pop_2035_Chicago_county.drop([0, 8, 9, 10, 11, 12]).rename(columns={'Head': 'County', '2035' : '2035pop'}).reset_index(drop=True)

In [12]:
df_pop_2035_Indiana_county = df_pop_2035_Indiana_county_input.iloc[:, 0:2]
df_pop_2035_Indiana_county = df_pop_2035_Indiana_county.drop(0).rename(columns={'Unnamed: 0': 'County', 'Unnamed: 1': '2035pop'})
df_pop_2035_Indiana_county = df_pop_2035_Indiana_county.assign(Fullname = df_pop_2035_Indiana_county.County.astype(str) + ', IN').reset_index(drop=True)

In [13]:
df_pop_2035_Minnesota_county = df_pop_2035_Minnesota_county_input.loc[(df_pop_2035_Minnesota_county_input.year == 2035) & (df_pop_2035_Minnesota_county_input.geo_type > 2)]
df_pop_2035_Minnesota_county = df_pop_2035_Minnesota_county[['geo_id', '999']].rename(columns={'geo_id': 'County', '999': '2035pop'})
df_pop_2035_Minnesota_county = df_pop_2035_Minnesota_county.assign(Fullname = df_pop_2035_Minnesota_county.County.astype(str) + ' County, MN').reset_index(drop=True)

In [14]:
df_pop_2035_Michigan_county = df_pop_2035_Michigan_county_input[['Area', '2035']]
df_pop_2035_Michigan_county = df_pop_2035_Michigan_county.rename(columns={'Area': 'Fullname', '2035': '2035pop'}).reset_index(drop=True)
df_pop_2035_Michigan_county = df_pop_2035_Michigan_county.assign(County = df_pop_2035_Michigan_county.Fullname.str[:-4])

In [15]:
df_pop_2035_Wisconsin_county_input.columns = df_pop_2035_Wisconsin_county_input.iloc[1]
df_pop_2035_Wisconsin_county = df_pop_2035_Wisconsin_county_input[['County Name', 'PopAll 2035']].drop([0, 1])
df_pop_2035_Wisconsin_county = df_pop_2035_Wisconsin_county.assign(Fullname = df_pop_2035_Wisconsin_county[['County Name']].astype(str) + 'County, WI')
df_pop_2035_Wisconsin_county = df_pop_2035_Wisconsin_county.assign(Fullname = df_pop_2035_Wisconsin_county.Fullname.str[1:])
df_pop_2035_Wisconsin_county = df_pop_2035_Wisconsin_county.rename(columns={'County Name': 'County', 'PopAll 2035': '2035pop'}).reset_index(drop=True)

In [16]:
df_pop_2035_county_input = pd.concat([df_pop_2035_Chicago_county, df_pop_2035_Indiana_county, df_pop_2035_Minnesota_county, df_pop_2035_Michigan_county, df_pop_2035_Wisconsin_county], ignore_index=True)
df_pop_2035_county_input.Fullname = df_pop_2035_county_input.Fullname.str.replace('Saint', 'St.', regex=True) # CAUTION!
df_pop_2035_county = pd.merge(df_pop_2035_county_input, df_countylist_input, on = 'Fullname')
df_pop_2035_county = df_pop_2035_county[['ZoneID','Fullname','2035pop']]
df_pop_2035_county.to_csv('Output_2035pop_WisDOT.csv')